In [1]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding



C:\Users\abhim\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
## Plotly
#import plotly.offline as py
#import plotly.graph_objs as go
#py.init_notebook_mode(connected=True)

# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE

In [3]:
data = pd.read_csv('Reviews.csv')

In [4]:
data.shape

(568454, 10)

In [5]:
data.tail(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...
568453,568454,B001LR2CU2,A3LGQPJCZVL9UC,srfell17,0,0,5,1338422400,Great Honey,"I am very satisfied ,product is as advertised,..."


In [6]:
df = data.head(data.shape[0])

In [7]:
df.shape

(568454, 10)

In [8]:
df.dropna(inplace=True)
df[df['Score'] != 3]
df['Positivity'] = np.where(df['Score'] > 3, 1, 0)
df.head()

C:\Users\abhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\abhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Positivity
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [9]:
df.shape

(568411, 11)

In [10]:
df.reset_index(inplace = True)

In [11]:
df.index

RangeIndex(start=0, stop=568411, step=1)

In [12]:
df.head(1)

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Positivity
0,0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1


In [13]:
### Text Normalizing function. Part of the following function was taken from this link. 

def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text





In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\a

[nltk_data]    |   Package qc is already up-to-date!
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]    |     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     C:\Users\ab

True

In [16]:
from nltk.stem.snowball import SnowballStemmer

In [17]:
from nltk.stem import PorterStemmer

In [18]:
nltk.download('SnowballStemmer')

[nltk_data] Error loading SnowballStemmer: Package 'SnowballStemmer'
[nltk_data]     not found in index


False

In [19]:
import re

In [20]:
## apply the above function to df['text']

df['Text'] = df['Text'].map(lambda x: clean_text(x))

C:\Users\abhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
df['Text'][123256]

'tast great starbuck coffe ! ! love item dwish run faster sugar plus version ! !'

In [22]:
list(df['Text'][1])

['p',
 'r',
 'o',
 'd',
 'u',
 'c',
 't',
 ' ',
 'a',
 'r',
 'r',
 'i',
 'v',
 ' ',
 'l',
 'a',
 'b',
 'e',
 'l',
 ' ',
 'j',
 'u',
 'm',
 'b',
 'o',
 ' ',
 's',
 'a',
 'l',
 't',
 ' ',
 'p',
 'e',
 'a',
 'n',
 'u',
 't',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'p',
 'e',
 'a',
 'n',
 'u',
 't',
 ' ',
 'a',
 'c',
 't',
 'u',
 'a',
 'l',
 ' ',
 's',
 'm',
 'a',
 'l',
 'l',
 ' ',
 's',
 'i',
 'z',
 'e',
 ' ',
 'u',
 'n',
 's',
 'a',
 'l',
 't',
 ' ',
 's',
 'u',
 'r',
 'e',
 ' ',
 'e',
 'r',
 'r',
 'o',
 'r',
 ' ',
 'v',
 'e',
 'n',
 'd',
 'o',
 'r',
 ' ',
 'i',
 'n',
 't',
 'e',
 'n',
 'd',
 ' ',
 'r',
 'e',
 'p',
 'r',
 'e',
 's',
 ' ',
 'p',
 'r',
 'o',
 'd',
 'u',
 'c',
 't',
 ' ',
 'j',
 'u',
 'm',
 'b',
 'o']

In [23]:
### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['Text'])

sequences = tokenizer.texts_to_sequences(df['Text'])
data = pad_sequences(sequences, maxlen=60)

In [24]:
type(data); data.shape; data[200000]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    9,    3,  135,   65,  335,  151,  623,
        700,    2, 3498,  718,  177,  860,   32,  121,  107,  282,  919,
        134,  965,   10,  314,   10, 4113, 8393,   65,  483,   61,  858,
       2415,  135,  477, 9594, 3424])

In [25]:
labels = df['Positivity']

In [26]:
type(labels); labels.shape

(568411,)

In [27]:
## Network architecture

model = Sequential()
model.add(Embedding(20000, 100, input_length=60))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
## Fit the model

model.fit(data, np.array(labels), validation_split=0.4, epochs=3)

Train on 341046 samples, validate on 227365 samples
Epoch 1/3
341046/341046 [==============================] - 4546s 13ms/step - loss: 0.2876 - acc: 0.8790 - val_loss: 0.2455 - val_acc: 0.8993
Epoch 2/3
341046/341046 [==============================] - 1475s 4ms/step - loss: 0.2302 - acc: 0.9062 - val_loss: 0.2308 - val_acc: 0.9076
Epoch 3/3
341046/341046 [==============================] - 1415s 4ms/step - loss: 0.1986 - acc: 0.9210 - val_loss: 0.2234 - val_acc: 0.9136


In [29]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [30]:
model.save_weights('model.h5')

In [31]:
model.load_weights('model.h5', by_name=False)

In [32]:
for layer in model.layers:
    weights = layer.get_weights() # list of numpy arrays

In [33]:
type(weights); len(weights)

2

In [34]:
len(weights[0]); weights

[array([[-9.5181726e-03],
        [ 1.7322856e-01],
        [-3.9306688e-01],
        [-2.7415580e-01],
        [ 3.5360891e-01],
        [-9.9213265e-02],
        [-1.4074652e-01],
        [ 2.1374597e-01],
        [ 1.0873618e-01],
        [-1.6799006e-01],
        [ 3.2996047e-02],
        [ 3.2462385e-01],
        [ 1.6775368e-01],
        [ 9.2461400e-02],
        [ 4.2036104e-01],
        [-2.4957187e-04],
        [-6.5839052e-02],
        [-5.9989844e-02],
        [ 3.4190226e-01],
        [-2.8219673e-01],
        [ 3.5590887e-01],
        [-1.9900855e-03],
        [ 2.1934643e-01],
        [ 2.3801403e-01],
        [ 2.6073623e-01],
        [-3.5452890e-01],
        [ 1.3177317e-01],
        [ 1.0934023e-01],
        [-4.6667421e-01],
        [-1.8484534e-01],
        [-4.6169326e-01],
        [-5.3314990e-01],
        [ 2.4868283e-01],
        [ 3.5500118e-01],
        [ 4.6863291e-01],
        [ 2.9631463e-01],
        [-3.9658549e-01],
        [ 3.6647519e-01],
        [ 1.

In [35]:
#Get embedding weights from the glove
word_embds = model.layers[0].get_weights()[0]

In [36]:
# Get word list
word_list = []
for word, i in tokenizer.word_index.items():
    word_list.append(word)

In [37]:
X_embedded = TSNE(n_components=2).fit_transform(word_weights)

number_of_words = 1000
trace = go.Scatter(
    x = X_embedded[0:number_of_words,0], 
    y = X_embedded[0:number_of_words, 1],
    mode = 'markers',
    text= word_list[0:number_of_words]
)

layout = dict(title= 't-SNE 1 vs t-SNE 2 for sirst 1000 words ',
              yaxis = dict(title='t-SNE 2'),
              xaxis = dict(title='t-SNE 1'),
              hovermode= 'closest')

fig = dict(data = [trace], layout= layout)
py.iplot(fig)

NameError: name 'word_weights' is not defined